In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Importing and processing the data into a dataloader
re_size = 35
crop_size = 32
data_transforms =  transforms.Compose([
        transforms.Resize([re_size,]),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor()
    ])

imageData = torchvision.datasets.ImageFolder(root = '/home/cmf290/dr/data/unhealthy',transform=data_transforms)
train_data, val_data, test_data = torch.utils.data.random_split(imageData,[0.6,0.2,0.2])
dataLoader_train = DataLoader(train_data, batch_size=64,shuffle=True)
dataLoader_val = DataLoader(val_data, batch_size=64,shuffle=False)

In [5]:
#Instantiate feature extractors and freeze weights
resnet18 = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in resnet18.parameters():
    param.requires_grad = False
    
resnet34 = torchvision.models.resnet34(weights='IMAGENET1K_V1')
for param in resnet34.parameters():
    param.requires_grad = False
    
resnet50 = torchvision.models.resnet50(weights='IMAGENET1K_V1')
for param in resnet50.parameters():
    param.requires_grad = False

resnet101 = torchvision.models.resnet101(weights='IMAGENET1K_V1')
for param in resnet101.parameters():
    param.requires_grad = False
    
resnet152 = torchvision.models.resnet152(weights='IMAGENET1K_V1')
for param in resnet152.parameters():
    param.requires_grad = False

densenet121 = torchvision.models.densenet121(weights='IMAGENET1K_V1')
for param in densenet121.parameters():
    param.requires_grad = False

densenet161 = torchvision.models.densenet161(weights='IMAGENET1K_V1')
for param in densenet161.parameters():
    param.requires_grad = False

densenet169 = torchvision.models.densenet169(weights='IMAGENET1K_V1')
for param in densenet169.parameters():
    param.requires_grad = False

densenet201 = torchvision.models.densenet201(weights='IMAGENET1K_V1')
for param in densenet201.parameters():
    param.requires_grad = False

vgg11 = torchvision.models.vgg11(weights='IMAGENET1K_V1')
for param in vgg11.parameters():
    param.requires_grad = False

vgg13 = torchvision.models.vgg13(weights='IMAGENET1K_V1')
for param in vgg13.parameters():
    param.requires_grad = False

vgg16 = torchvision.models.vgg16(weights='IMAGENET1K_V1')
for param in vgg16.parameters():
    param.requires_grad = False

vgg19 = torchvision.models.vgg19(weights='IMAGENET1K_V1')
for param in vgg19.parameters():
    param.requires_grad = False

inception_v3 = torchvision.models.inception_v3(weights='IMAGENET1K_V1')
for param in inception_v3.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\colef/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:10<00:00, 8.07MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\colef/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:13<00:00, 7.42MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\colef/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:25<00:00, 6.97MB/s] 
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\colef/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:35<00:00, 6.76MB/s] 
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\colef/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:06<00:00, 5.29MB/s]
Downloading: "htt

In [18]:
#List of names for feature extractor models used instantiated in the above cell
models = list([resnet18,resnet34,resnet50,resnet101,resnet152,densenet121,densenet161,densenet169,densenet201])

In [9]:
#Feature extractor, concatenation, and fully connected neural network with one hidden layer
class Feature_Extractor_ClassifierNetwork(nn.Module):
    def __init__(self, models, hidden_size, num_classes):
        super(Feature_Extractor_ClassifierNetwork, self).__init__()

        assert type(models) == list and (not len(models) == 0)

        self.feature_models = []
        self.input_filters = 0
        self.hidden_size = hidden_size

        #loop through the models in the model list defined in above cell
        for i, model in enumerate(models):
          #define layers of each feature extrator model using list(model.childen())
          layers = list(model.children())
          #count the number of features extracted right before the classification layer *hint: layers[-1].in_features
          feature_count = layers[-1].in_features
          #add the number of output features together for everymodel and store in input_filters
          self.input_filters += feature_count
          #append the feature extractor model (without the classifier head to feature_models hint *nn.Sequential(*list(model.children())[:-1]))
          self.feature_models.append(nn.Sequential(*list(model.children())[:-1]))

        # define first fully connected layer with input as input_filters and output as hidden_size)
        self.first_layer = nn.Linear(self.input_filters,hidden_size)
        # define hyperbolic tangent activation
        self.hyperbolic_tan = nn.Tanh()
        # define second fully connected layer with hidden size input and number of classes as output
        self.second_layer = nn.Linear(hidden_size,num_classes)


    def forward(self, x):
      # define an empty tensor to store the concatenated features
      concatenated_features = torch.Tensor()
      # loop through each feature model in feature_model define above
      for i, model in enumerate(self.feature_models):
        # feed your input images, x, through the feature extractors then concatenate output of all the models into feat_concat
        output = model(x)
        output = output[:,:,0,0]
        # *warning: concatenate horizontally be mindful of axis of concatenation
        concatenated_features = torch.cat((concatenated_features,output),dim=1)

      # feed concatenated features into first  fully connect layer
      output = self.first_layer(concatenated_features)
      # feed output from first  fully connect layer into hyp tan activation
      output = self.hyperbolic_tan(output)
      # feed output of the activation into second fully connect layer
      output = self.second_layer(output)
      return output

In [14]:
type(models)

list

In [19]:
#Check model runs with a random tensor
test_model = Feature_Extractor_ClassifierNetwork(models,1024,5)
x=torch.randn(4,3,32,32)
test_model(x)

tensor([[-0.0555,  0.5938, -0.1299, -0.2065, -0.0646],
        [ 0.0268, -0.1605, -0.1201, -0.1783, -0.0557],
        [ 0.1819,  0.0191, -0.2334, -0.2857,  0.0391],
        [-0.1429,  0.2425, -0.0398, -0.2031, -0.0804]],
       grad_fn=<AddmmBackward0>)

In [ ]:
# Train the model on your optimization/training loop designed previously with cross-entropy loss
def train(model, optimizer, train_dataloader, val_dataloader, criterion):
    transformation = transforms.ToTensor()
    #set model to train mode
    model.train()
    #initialize training varaibles to track, i.e. loss and accuracy
    running_loss = 0.
    running_accuracy = 0.
    #enumeration loop
    for i, data in enumerate(train_dataloader):
        # X, Y = X.to(device), Y.to(device)
        inputs, labels = data
        new_labels = []
        for idx, label in enumerate(labels):
            adjusted_label = [0.,0.,0.,0.,0.]
            adjusted_label[label] = 1.
            adjusted_label = np.float32(np.array(adjusted_label))
            new_labels.append(adjusted_label)
        labels = new_labels
        labels = transformation(np.array(labels))
        labels = np.transpose(labels,[1,-1,0])
        labels = labels[:,:,0]
        #clear the grad of each parameter
        optimizer.zero_grad()
        #forward pass
        outputs = model(inputs)
        #compute loss use functional loss: nnF.mse_loss
        loss = criterion(outputs,labels)
        #backward pass
        loss.backward()
        #update parameters with optimizer
        optimizer.step()
        #record the loss with the loss variable
        running_loss += loss.item()
        #convert prediction to int for accuracy computation.
        for idx, out_data in enumerate(outputs):
            max_idx = torch.argmax(out_data)
            outputs[idx] = torch.tensor([0.,0.,0.,0.,0.])
            outputs[idx][max_idx] = 1.
        # #measure accuracy on this batch and sum to accuracy variable

        correct = (outputs == labels).float().sum() / 5


        running_accuracy += correct

    val_running_loss = 0
    val_running_accuracy = 0

    for i, data in enumerate(val_dataloader):
        # X, Y = X.to(device), Y.to(device)
        inputs, labels = data
        new_labels = []
        for idx, label in enumerate(labels):
            adjusted_label = [0.,0.,0.,0.,0.]
            adjusted_label[label] = 1.
            adjusted_label = np.float32(np.array(adjusted_label))
            new_labels.append(adjusted_label)
        labels = new_labels
        labels = transformation(np.array(labels))
        labels = np.transpose(labels,[1,-1,0])
        labels = labels[:,:,0]
        #clear the grad of each parameter
        optimizer.zero_grad()
        #forward pass
        outputs = model(inputs)
        #compute loss use functional loss: nnF.mse_loss
        loss = criterion(outputs,labels)
        #record the loss with the loss variable
        val_running_loss += loss.item()
        #convert prediction to int for accuracy computation.
        for idx, out_data in enumerate(outputs):
            max_idx = torch.argmax(out_data)
            outputs[idx] = torch.tensor([0.,0.,0.,0.,0.])
            outputs[idx][max_idx] = 1.
        # #measure accuracy on this batch and sum to accuracy variable

        correct = (outputs == labels).float().sum() / 5


        val_running_accuracy += correct


    #compute average loss and accuracy
    loss_train = running_loss / len(train_dataloader)
    accuracy_train = running_accuracy / len(train_dataloader.dataset)
    loss_val = val_running_loss / len(val_dataloader)
    accuracy_val = val_running_accuracy / len(val_dataloader.dataset)
    #return accuracy and loss
    return loss_train, accuracy_train, loss_val, accuracy_val

In [ ]:
#Instantiate model, optimizer, and criterion and train
model = Feature_Extractor_ClassifierNetwork(models,1024,5)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = torch.nn.CrossEntropyLoss()

loss_train_list=[]
acc_train_list=[]
loss_val_list = []
acc_val_list = []

for epoch in range(0, 3):
    #-------- perform training --------------------------------
    loss_train, acc_train, loss_val, acc_val = train(model,optimizer,dataLoader_train,dataLoader_val,criterion)
    loss_train_list.append(loss_train)
    acc_train_list.append(acc_train)
    loss_val_list.append(loss_val)
    acc_val_list.append(acc_val)

    print('epoch', epoch, 'training loss:', loss_train, 'acc:', acc_train)

In [ ]:
#Plot the loss and accuracy results
fig, ax = plt.subplots(1, 2, figsize=(12,6))
ax[0].plot(np.arange(0,len(loss_train_list)), loss_train_list, '-b', label='training loss')
ax[0].plot(np.arange(0,len(loss_val_list)), loss_val_list, '-g', label='validation loss')
ax[0].set_xlabel('epoch',fontsize=16)
ax[0].legend(fontsize=16)
ax[0].grid(True)
ax[1].plot(np.arange(0,len(acc_train_list)), acc_train_list, '-b', label='training accuracy')
ax[1].plot(np.arange(0,len(acc_val_list)), acc_val_list, '-g', label='validation accuracy')
ax[1].set_xlabel('epoch',fontsize=16)
ax[1].legend(fontsize=16)
ax[1].grid(True)